Before you turn this problem in, make sure everything runs as expected. 
  1. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and 
  2. Then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name below. 

---

# CSCI 3155: Assignment 8

Goals of this assignment include: 
- Developing a type system for references
- Understanding the limitations of type systems using the division by zero example
- A simple exercise in lazy functional programming

__Name__: WRITE YOUR NAME HERE

In [26]:
// TEST HELPER
def passed(points: Int) {
    require(points >=0)
    if (points == 1) print(s"\n*** Tests Passed (1 point) ***\n")
    else print(s"\n*** Tests Passed ($points points) ***\n")
}

defined function passed

## Problem 1 (25 points)

In this problem, we will extend our type system for handling references.  Here is a stripped down version of Lettuce that involves references.

$$\begin{array}{rcl}
\mathbf{Expr} & \rightarrow & Const(\mathbf{Double}) \\
& | & Ident(\mathbf{Identifier})\\
& | & Plus(\mathbf{Expr}, \mathbf{Expr})\\
& | & Div(\mathbf{Expr}, \mathbf{Expr}) \\
& | & Geq(\mathbf{Expr}, \mathbf{Expr}) \\
& | & And(\mathbf{Expr}, \mathbf{Expr}) \\
& | & IfThenElse(\mathbf{Expr}, \mathbf{Expr}, \mathbf{Expr})\\
& | & Let(\mathbf{Identifier}, \color{red}{\mathbf{Type}}, \mathbf{Expr}, \mathbf{Expr})\\
& | & FunDef(\mathbf{Identifier}, \color{red}{\mathbf{Type}}, \mathbf{Expr}) \\
& | & FunCall(\mathbf{Expr}, \mathbf{Expr}) \\
& | & \color{blue}{NewRef(\mathbf{Expr})} \\
& | & \color{blue}{DeRef(\mathbf{Expr})} \\
& | & \color{blue}{AssignRef(\mathbf{Expr}, \mathbf{Expr})} \\
\end{array}$$

We have a type system 
$$\begin{array}{rcl}
\mathbf{Type} & \rightarrow & NumType & \leftarrow \text{written as } \ num \\
& | & BoolType & \leftarrow \text{written as } \ bool \\
& | & FunType(\mathbf{Type} , \mathbf{Type} ) & \leftarrow \text{written as } \ t_1 \Rightarrow t_2 \\
& | & \color{red}{RefType(\mathbf{Type})} & \leftarrow \text{written as } \ ref(t_1) \\
\end{array}$$


### A (5 points)
Fill out the missing type annotations labeled type1, type2, and type3 in the program below.

``` ocaml
let x: type1 = 25 in 
 let y : type2 = NewRef( - x ) in 
  let f : type3 = NewRef(
                    function (z: ref(num)) DeRef(z) - x >= 45 
                   ) in 
   let z : ref(bool) = DeRef(f)(y) in 
      z
```

YOUR ANSWER HERE

type1:NumType

type2:RefType(NumType)

type3:RefType(FunType(RefType(NumType), BoolType))

### B (10 points): Let us write some inference rules for working with ref types.
$$\newcommand\typeOf{\mathbf{typeOf}}$$
$$\newcommand\semRule[3]{\begin{array}{c} #1 \\ \hline #2 \\ \end{array}\;(\text{#3}) }$$
Recall from notes on "Types and Type Checking" that $\typeOf(\texttt{e}, \alpha)$ is the type of an 
expression $\texttt{e}$ under type environment $\alpha$. The type environment maps identifiers in the current scope to  their annotated types.


Let us write a rule for NewRef.

$$\semRule{ \typeOf(\texttt{e}, \alpha) = t,\; t \not= \mathbf{typeerror} }{\typeOf(\texttt{NewRef(e)}, \alpha) = ref(t) }{newref-ok}$$

It says that if $\texttt{e}$ receives type $t$ under type environment $\alpha$ and it is not a type error, then $\texttt{NewRef(e)}$ must receive the type $ref(t)$ under $\alpha$.



(i) Complete the missing terms for the rule for `DeRef` OK rule.
$$\semRule{\typeOf(\texttt{e}, \alpha) = ref(t)}{\typeOf(\texttt{DeRef(e)}, \alpha) = \color{red}{???_1}}{deref-ok}$$

(ii) Complete the missing terms for the rule for `DeRef` error.
$$\semRule{\typeOf(\texttt{e}, \alpha) = t,\ t \not= \color{red}{???_2} }{\typeOf(\texttt{DeRef(e)}, \alpha) = \mathbf{typeerror}}{deref-nok}$$

(iii) Complete the missing terms for `AssignRef` OK rule.
$$\semRule{\typeOf(\texttt{e1}, \alpha) = \color{red}{???_3}, \typeOf(\texttt{e2}, \alpha) = t}{\typeOf(\texttt{AssignRef(e1, e2)}, \alpha) =   \color{red}{???_4}}{assignref-ok}$$


YOUR ANSWER HERE

1.t

2.ref(t)

3.ref(t)

4.t

### C (10 points): Write the type checker with references by filling in the missing code.

In [27]:
sealed trait Type
case object NumType extends Type
case object BoolType extends Type
case class FunType(t1: Type, t2: Type) extends Type
// TODO: Write a new case class for RefType
// YOUR CODE HERE
case class RefType(t1: Type) extends Type

sealed trait Program
sealed trait Expr

case class Const(f: Double) extends Expr
case class Ident(s: String) extends Expr
case class Plus(e1: Expr, e2: Expr) extends Expr
case class Minus(e1: Expr, e2: Expr) extends Expr
case class Geq(e1: Expr, e2: Expr) extends Expr
case class IfThenElse(e1: Expr, e2: Expr, e3: Expr) extends Expr
case class Let(x: String, xType: Type, e1: Expr, e2: Expr) extends Expr
case class FunDef(id: String, idType: Type, e: Expr) extends Expr
case class FunCall(calledFun: Expr, argExpr: Expr) extends Expr
case class NewRef(e: Expr) extends Expr
case class DeRef(e: Expr) extends Expr
case class AssignRef(e1: Expr, e2: Expr) extends Expr
case class TopLevel(e: Expr) extends Program

def typeEquals(t1: Type, t2: Type): Boolean = t1 == t2
case class TypeErrorException(s: String) extends Exception


defined trait Type
defined object NumType
defined object BoolType
defined class FunType
defined class RefType
defined trait Program
defined trait Expr
defined class Const
defined class Ident
defined class Plus
defined class Minus
defined class Geq
defined class IfThenElse
defined class Let
defined class FunDef
defined class FunCall
defined class NewRef
defined class DeRef
defined class AssignRef
defined class TopLevel
defined function typeEquals
defined class TypeErrorException

In [28]:
def typeOf(e: Expr, alpha: Map[String, Type]): Type = {
    def checkType(opName: String, e1: Expr, t1: Type, e2: Expr, t2: Type, resType: Type): Type = {
        val t1hat = typeOf(e1, alpha)
        if (! typeEquals(t1hat, t1)){
            throw new TypeErrorException(s"Type mismatch in arithmetic/comparison/bool op $opName, Expected type $t1, obtained $t1hat")
        }
        
        val t2hat = typeOf(e2, alpha)
        if (! typeEquals(t2hat, t2)){
            throw new TypeErrorException(s"Type mismatch in arithmetic/comparison/bool op $opName, Expected type $t2, obtained $t2hat")
        }
        
        resType
    }
    
    e match {
        case Const(f) => NumType
        case Ident(s) => {if (alpha contains s)
                             alpha(s)
                          else 
                             throw TypeErrorException(s"Unknown identifier $s")}
        case Plus(e1, e2) =>  checkType("Plus", e1,  NumType, e2, NumType, NumType)
        case Minus(e1, e2) => checkType("Minus",e1,  NumType, e2, NumType, NumType)
        case Geq(e1, e2) => checkType("Geq", e1,  NumType, e2, NumType, BoolType)
        case IfThenElse(e, e1, e2) => {
            val t = typeOf(e, alpha)
            if (t == BoolType){
                val t1 = typeOf(e1, alpha)
                val t2 = typeOf(e2, alpha)
                if (typeEquals(t1, t2))
                    t1
                else 
                    throw TypeErrorException(s"If then else returns unequal types $t1 and $t2")
            } else {
                throw TypeErrorException(s"If then else condition expression not boolean $t")
            }
        }

        case Let(x, t, e1, e2) => {
            val t1 = typeOf(e1, alpha)
            if (typeEquals(t1, t)){
                val newAlpha = alpha + (x -> t)
                typeOf(e2, newAlpha)
            } else {
                throw TypeErrorException(s"Let binding has type $t whereas it is bound to expression of type $t1")
            }
        }

        case FunDef(x, t1, e) => {
            val newAlpha = alpha + (x -> t1)
            val t2 = typeOf(e, newAlpha)
            FunType(t1, t2)
        }

        case FunCall(e1, e2) => {
            val ftype = typeOf(e1, alpha)
            ftype match {
                case FunType(t1, t2) => {
                    val argType = typeOf(e2, alpha)
                    if (typeEquals(argType, t1)){
                        t2
                    } else {
                        throw TypeErrorException(s"Call to function with incompatible argument type. Expected $t1, obtained $argType")
                    }
                }
                case _ => { throw TypeErrorException(s"Call to function but with a non function type $ftype")}

            }
        }

        case NewRef(e) => {
            // YOUR CODE HERE
            //we have alpha in env,2 type t1,t2 follow above
            RefType(typeOf(e,alpha))
        }
        
        case AssignRef(e1, e2) => {
            // YOUR CODE HERE
            //assign value what e2 point to, to e1
             val temptype=typeOf(e1,alpha)
              val temptype2=typeOf(e2,alpha)
            temptype match
            {
                case RefType(temptype2)=>{temptype2 }
                case _ =>{throw TypeErrorException(s"failed") }
            }
        }
        
        case DeRef(e) => {
            // YOUR CODE HERE
            //get the value using deref
             val temptype=typeOf(e,alpha)
             temptype match
            {
               case RefType(t)=>{t}
             case _ =>{throw TypeErrorException(s"failed") }
            }
        }
        
    }
}

def typeOfProgram(p: Program) = p match {
    case TopLevel(e) => {
            val t = typeOf(e, Map())
            println(s"Program type computed successfully as $t")
            t
    }
}

defined function typeOf
defined function typeOfProgram

In [29]:
//BEGIN TEST

/* 
let x : ref(num) = NewRef(10 ) in 
   let dummy: num = AssignRef(x, 30) in 
       DeRef(x)
       */

val p1 = Let("x", RefType(NumType), NewRef(Const(10)), Let("dummy", NumType, AssignRef(Ident("x"), Const(30) ), DeRef(Ident("x"))) )
val t1 = typeOfProgram(TopLevel(p1))
assert(t1 == NumType, "Test 1 failed: answer should be NumType")
passed(2)
//END TEST

Program type computed successfully as NumType

*** Tests Passed (2 points) ***


p1: Let = Let(
  "x",
  RefType(NumType),
  NewRef(Const(10.0)),
  Let("dummy", NumType, AssignRef(Ident("x"), Const(30.0)), DeRef(Ident("x")))
)
t1: Type = NumType

In [30]:
//BEGIN TEST
/* 
let x : ref(num) = NewRef(function(z: num) z + 10) in 
   let dummy: num = AssignRef(x, 30) in 
       DeRef(x)
       */
val fdef = FunDef("z", NumType, Plus(Ident("z"), Const(10)))
val p2 = Let("x", RefType(NumType), NewRef(fdef), Let("dummy", NumType, AssignRef(Ident("x"), Const(30) ), DeRef(Ident("x"))) )
val t2 = try {
   typeOfProgram(TopLevel(p2))
   assert(false, "The program should not receive a type")
} catch {
    case TypeErrorException(msg) => s"OK -- caught a type error exception: $msg"
    case e => print(e); assert(false, "Please throw TypeErrorException(message) when a type failure occurs")
}
passed(2)
//END TEST


*** Tests Passed (2 points) ***


fdef: FunDef = FunDef("z", NumType, Plus(Ident("z"), Const(10.0)))
p2: Let = Let(
  "x",
  RefType(NumType),
  NewRef(FunDef("z", NumType, Plus(Ident("z"), Const(10.0)))),
  Let("dummy", NumType, AssignRef(Ident("x"), Const(30.0)), DeRef(Ident("x")))
)
t2: Any = "OK -- caught a type error exception: Let binding has type RefType(NumType) whereas it is bound to expression of type RefType(FunType(NumType,NumType))"

In [31]:
//BEGIN TEST
/* 
let x : ref(num => num) = NewRef(function(z: num) z + 10) in 
   let dummy: num = AssignRef(x, 30) in 
       DeRef(x)
       */
val fdef = FunDef("z", NumType, Plus(Ident("z"), Const(10)))
val p3 = Let("x", RefType(FunType(NumType, NumType)), NewRef(fdef), Let("dummy", NumType, AssignRef(Ident("x"), Const(30) ), DeRef(Ident("x"))) )
val t3 = try {
   typeOfProgram(TopLevel(p3))
   assert(false, "The program should not receive a type")
} catch {
    case TypeErrorException(msg) => s"OK -- caught a type error exception: $msg"
    case e => print(e); assert(false, "Please throw TypeErrorException(message) when a type failure occurs")
}
passed(2)
//END TEST


*** Tests Passed (2 points) ***


fdef: FunDef = FunDef("z", NumType, Plus(Ident("z"), Const(10.0)))
p3: Let = Let(
  "x",
  RefType(FunType(NumType, NumType)),
  NewRef(FunDef("z", NumType, Plus(Ident("z"), Const(10.0)))),
  Let("dummy", NumType, AssignRef(Ident("x"), Const(30.0)), DeRef(Ident("x")))
)
t3: Any = "OK -- caught a type error exception: Let binding has type NumType whereas it is bound to expression of type FunType(NumType,NumType)"

In [32]:
//BEGIN TEST
/* 
let x : ref(num => num) = NewRef(function(z: num) z + 10) in 
   let dummy: num = AssignRef(NewRef(35), 30) in 
       DeRef(x)
       */
val fdef = FunDef("z", NumType, Plus(Ident("z"), Const(10)))
val p4 = Let("x", RefType(FunType(NumType, NumType)), NewRef(fdef), Let("dummy", NumType, AssignRef(NewRef(Const(35)), Const(30) ), DeRef(Ident("x"))) )
val t4 =  typeOfProgram(TopLevel(p4))
assert(t4 == FunType(NumType, NumType), "Test failed")
passed(2)
//END TEST

Program type computed successfully as FunType(NumType,NumType)

*** Tests Passed (2 points) ***


fdef: FunDef = FunDef("z", NumType, Plus(Ident("z"), Const(10.0)))
p4: Let = Let(
  "x",
  RefType(FunType(NumType, NumType)),
  NewRef(FunDef("z", NumType, Plus(Ident("z"), Const(10.0)))),
  Let(
    "dummy",
    NumType,
    AssignRef(NewRef(Const(35.0)), Const(30.0)),
    DeRef(Ident("x"))
  )
)
t4: Type = FunType(NumType, NumType)

In [33]:
//BEGIN TEST

val y = Ident("y")
val x = Ident("x")
val z = Ident("z")
val t1 = RefType(NumType)
val t2 = FunType(t1, NumType)
val t3 = RefType(t2)

val v1 = FunCall(DeRef(y), x)
val v2 = Let("z", RefType(BoolType), NewRef(Geq(DeRef(x), Const(25))), v1)
val fdef = FunDef("z", RefType(NumType), Plus(DeRef(z), DeRef(x)))
val v3 = Let("y", t3, NewRef(fdef), v2)
val v4 = Let("x", t1, NewRef(Const(10)), v3)

val p5 = TopLevel(v4)
assert(typeOfProgram(p5) == NumType, "Test failed")
passed(2)

//END TEST

Program type computed successfully as NumType

*** Tests Passed (2 points) ***


y: Ident = Ident("y")
x: Ident = Ident("x")
z: Ident = Ident("z")
t1: RefType = RefType(NumType)
t2: FunType = FunType(RefType(NumType), NumType)
t3: RefType = RefType(FunType(RefType(NumType), NumType))
v1: FunCall = FunCall(DeRef(Ident("y")), Ident("x"))
v2: Let = Let(
  "z",
  RefType(BoolType),
  NewRef(Geq(DeRef(Ident("x")), Const(25.0))),
  FunCall(DeRef(Ident("y")), Ident("x"))
)
fdef: FunDef = FunDef(
  "z",
  RefType(NumType),
  Plus(DeRef(Ident("z")), DeRef(Ident("x")))
)
v3: Let = Let(
  "y",
  RefType(FunType(RefType(NumType), NumType)),
  NewRef(
    FunDef("z", RefType(NumType), Plus(DeRef(Ident("z")), DeRef(Ident("x"))))
  ),
  Let(
    "z",
    RefType(BoolType),
    NewRef(Geq(DeRef(Ident("x")), Const(25.0))),
    FunCall(DeRef(Ident("y")), Ident("x"))
  )
)
v4: Let = Let(
  "x",
  RefType(NumType),
  NewRef(Const(10.0)),
  Let(
    "y",
    RefType(FunType(RefType(NumType), NumType)),
    NewRef(
      FunDef("z", RefType(NumType), Plus(DeRef(Ident("z")), DeRef(Ident(

## Problem 2 (10 points): Attempted Type System for Division by Zero

In class, we mentioned that developing a __static type system__ for catching run time errors is much harder. Let us try and see what happens.

Let us modify our type grammar to have two different types of numbers: zero and non-zero.
$$\newcommand\NonZeroNumType{\textsf{NonZeroNumType}}$$
$$\newcommand\ZeroNumType{\textsf{ZeroNumType}}$$

$$\begin{array}{rcl} 
\mathbf{Type} & \rightarrow & \NonZeroNumType \\
& \rightarrow & \ZeroNumType \\
\end{array}$$

with the following subset of Lettuce just involving arithmetic and let bindings:

$$\begin{array}{rcl}
\mathbf{Expr} & \rightarrow & Const(\mathbf{Double}) \\
& | & Ident(\mathbf{Identifier})\\
& | & Plus(\mathbf{Expr}, \mathbf{Expr})\\
& | & Mult(\mathbf{Expr}, \mathbf{Expr}) \\
& | & Div(\mathbf{Expr}, \mathbf{Expr})\\
& | & Let(\mathbf{Identifier}, \color{red}{\mathbf{Type}}, \mathbf{Expr}, \mathbf{Expr})\\
\end{array}$$

__Goal:__ Guarantee that a numeric value that receives type _ZeroNumType_ must be $0$ and a numeric value that receives _NonZeroNumType_ must always be non-zero.

__Semantic Rules:__ 

__Constant:__ The new rule for constants are

$$\semRule{f \not= 0}{\typeOf(\texttt{Const(f)}, \alpha) = \NonZeroNumType}{const-non-zero}$$

$$\semRule{f = 0}{\typeOf(\texttt{Const(f)}, \alpha) = \ZeroNumType}{const-zero}$$

__Identifiers:__ The rule for identifiers remains unchanged (see notes).

__Plus:__ The new rules for plus are

$$\semRule{\typeOf(e1, \alpha) = \ZeroNumType,\ \typeOf(e2, \alpha) = \ZeroNumType}{\typeOf(\texttt{Plus(e1, e2)}, \alpha) = \ZeroNumType}{plus-zero}$$

$$\semRule{\typeOf(e1, \alpha) = \NonZeroNumType,\ \typeOf(e2, \alpha) = \ZeroNumType}{\typeOf(\texttt{Plus(e1, e2)}, \alpha) = \NonZeroNumType}{plus-non-zero-1}$$

$$\semRule{\typeOf(e1, \alpha) = \ZeroNumType,\ \typeOf(e2, \alpha) = \NonZeroNumType}{\typeOf(\texttt{Plus(e1, e2)}, \alpha) = \NonZeroNumType}{plus-non-zero-2}$$

$$\semRule{\typeOf(e1, \alpha) = \NonZeroNumType,\ \typeOf(e2, \alpha) = \NonZeroNumType}{\typeOf(\texttt{Plus(e1, e2)}, \alpha) = \NonZeroNumType}{plus-non-zero-3}$$


__Division:__ The rules for division are

$$\semRule{\typeOf(e1, \alpha) = \ZeroNumType,\ \typeOf(e2, \alpha) = \NonZeroNumType}{\typeOf(\texttt{Div(e1, e2)}, \alpha) = \ZeroNumType}{div-zero}$$

$$\semRule{\typeOf(e2, \alpha) = \ZeroNumType}{\typeOf(\texttt{Div(e1, e2)}, \alpha) = \mathbf{typeerror}}{div-error}$$

$$\semRule{\typeOf(e1, \alpha) = \NonZeroNumType,\ \typeOf(e2, \alpha) = \NonZeroNumType}{\typeOf(\texttt{Div(e1, e2)}, \alpha) = \NonZeroNumType}{div-ok}$$

__Let Binding:__ Rules are unchanged (see notes).


We claim that if a Lettuce program is annotated with the types ZeroNumType and NonZeroNumType, and the type annotations are according to the rules above, then any attempt to divide a number by zero will result in __typeerror__ rather than a runtime exception thrown when evaluating the program using an interpreter.

__(A)__ Complete the missing parts of the  semantic rules for multiplication below.

$$\semRule{\typeOf(e1, \alpha) = \color{red}{??_1}}{\typeOf(\texttt{Mult(e1, e2)}, \alpha) = \ZeroNumType}{mult-first-arg1}$$

$$\semRule{\typeOf(e1, \alpha) = \NonZeroNumType, \typeOf(e2, \alpha) = \color{red}{??_2}}{\typeOf(\texttt{Mult(e1, e2)}, \alpha) = \ZeroNumType}{mult-first-arg2}$$

$$\semRule{\typeOf(e1, \alpha) = \color{red}{??_3}, \typeOf(e2, \alpha) = \color{red}{??_4}}{\typeOf(\texttt{Mult(e1, e2)}, \alpha) = \NonZeroNumType}{mult-non-zero}$$



__(B)__ Write a Lettuce program that has the correct type annotations 
(you should use the concrete syntax for your answer) but results in a division by zero that is not 
detected as a __typeerror__.


__(C)__ Point out which rule among the ones provided is incorrect (techically an incorrect type rule is called unsound).


YOUR ANSWER HERE
__(A)__ 1.ZeroNumType 2.ZeroNumType 3.NonZeroNumType 4.NonZeroNumType

__(B)__ An example is Div(Const(14),Plus(Const(2),Const(-2)))

Because there is loopholes in plus-non-zero-3

__(C)__ plus-non-zero-3 is incorrect, it could be 2 nonzero-numtype add up to 0,then return ZeroNumtype.For example,-2 and 2 return ZeroNumtype

## Problem 3 A ( 5 points)
Create an infinite stream that generates all natural numbers starting from 0. Your stream must be a value called allNatural of type `Stream[Int]`

In [34]:
val allNatural:Stream[Int] = {
    // YOUR CODE HERE
   Stream.from(0)
}

allNatural: Stream[Int] = Stream(
  0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
...

In [35]:
//BEGIN TEST
def testStream(n: Int): Boolean = {
    (allNatural.take(n).toList == List.range(0, n))
}

assert(testStream(5), "Testing first 5 elements of the stream")
assert(testStream(0), "Testing first 0 elements of the stream")
assert(testStream(25), "Testing first 25 elements of the stream")
assert(testStream(55), "Testing first 55 elements of the stream")
assert(testStream(1303), "Testing first 1303 elements of the stream")
passed(10)
//END TEST


*** Tests Passed (10 points) ***


defined function testStream

## Problem 3B (10 points)

Write an infinite stream that will generate all natural numbers `n` that satisfy all of  the following condition: 
  - n mod 2 != 0
  - n mod 3 != 0
  - n mod 5 != 0
  - n mod 7 != 0
  - n mod 11 != 0
  - n mod 13 != 0
  - n mod 17 != 0
  - n mod 19 != 0
  
 Your solution should use the stream of all natural numbers created in the previous problem and a suitable application of `filter` on streams.

In [36]:
val specialNatNumberStream: Stream[Int] = {
    // YOUR CODE HERE
    allNatural.filter(_%2!=0).filter(_%3!=0).filter(_%5!=0)
    .filter(_%7!=0).filter(_%11!=0).filter(_%13!=0)
    .filter(_%17!=0).filter(_%19!=0)
}

specialNatNumberStream: Stream[Int] = Stream(
  1,
  23,
  29,
  31,
  37,
  41,
  43,
  47,
  53,
  59,
  61,
  67,
  71,
  73,
  79,
  83,
  89,
  97,
  101,
  103,
  107,
  109,
  113,
  127,
  131,
  137,
  139,
  149,
  151,
  157,
  163,
  167,
  173,
  179,
  181,
  191,
  193,
  197,
...

In [37]:
//BEGIN TEST
def cond(j: Int): Boolean = {
        j % 2 != 0 && j % 3 != 0 && j % 5 != 0 && 
        j % 7 != 0 && j % 11 != 0 && j % 13 != 0 && 
        j % 17 != 0 && j % 19 != 0
    }

def testStream(n: Int): Boolean = {
    specialNatNumberStream.take(n).forall( cond)
}

assert(testStream(2000), "First 2000 elements test failed")
passed(10)
//END TEST


*** Tests Passed (10 points) ***


defined function cond
defined function testStream

## That's all folks!